In [ ]:
%matplotlib notebook
%precision 3

from IPython.display import set_matplotlib_formats
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import matplotlib
import math
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [9.5, 5]
import pandas

In [ ]:
# source: https://scrippsco2.ucsd.edu/data/atmospheric_co2/mlo.html
co2s=pandas.read_csv("https://scrippsco2.ucsd.edu/assets/data/atmospheric/stations/in_situ_co2/weekly/weekly_in_situ_co2_mlo.csv", skiprows=44, header=0,names=["date", "co2ppm"])
co2s["timestamp"]=pandas.to_datetime(co2s["date"],infer_datetime_format=True)
co2s["doy"]=co2s.timestamp.dt.day_of_year
co2s.set_index("timestamp", inplace=True)


In [ ]:
co2s

In [ ]:
plt.figure()
plt.plot(co2s.co2ppm)
plt.plot(co2s.co2ppm.rolling(52, center=True).mean())
plt.title('Mauna Loa Observatory CO2 PPM')
plt.ylabel("ppm")
plt.grid()

In [ ]:
plt.figure()
absol=np.abs(co2s.co2ppm.rolling(52, center=True).mean() - co2s.co2ppm)
plt.plot(absol)
plt.plot(absol.rolling("365d").mean())
plt.title("Absolute deviation of Mauna Loa Observatory CO2 PPM from annualized data")
plt.ylabel("abs ppm")
plt.grid()

In [ ]:
plt.figure()
co2s["devia"] = co2s.co2ppm- co2s.co2ppm.rolling(52, center=True).mean()
plt.plot(co2s.devia)
plt.title("Deviation of Mauna Loa Observatory CO2 PPM from annualized data")
plt.ylabel("ppm")

plt.grid()

In [ ]:
clean=co2s.dropna()
clean

In [ ]:
part1=clean["2000-01-01":]
coeff1 = np.polyfit(part1.doy, part1.devia, 7)
part2=clean[:"2000-01-01"]
coeff2 = np.polyfit(part2.doy, part2.devia, 7)

days=np.linspace(0, 365, 52)

In [ ]:
plt.figure()
plt.scatter(part1.doy, part1.devia, s=0.1)
plt.scatter(part2.doy, part2.devia, s=0.1)
plt.plot(days, np.polyval(coeff1, days), label="Deviation <2000")
plt.plot(days, np.polyval(coeff2, days), label="Deviation >=2000")
plt.grid()
plt.legend()
plt.title("Deviation of annualized Mauna Loa Observatory CO2 PPM per day of year")

In [ ]:
plt.figure()
trend=co2s.co2ppm - np.polyval(coeff2, co2s.doy)
plt.plot(trend)
plt.plot(co2s.co2ppm, alpha=0.2, color='blue')
plt.title("Mauna Loa Observatory CO2 PPM with annual swing removed")
plt.ylabel("ppm")
plt.grid()

In [ ]:
plt.figure()
plt.plot(52*trend["1989-01-01":].diff().rolling(52).mean(), label="Lagging yearly average")
plt.plot(52*trend["1980-01-01":].diff().rolling(520).mean(), label="Lagging 10 year average")

plt.grid()
plt.ylabel("PPM/year")
plt.title("Yearly rate of increase of Mauna Loa Observatory CO2 PPM compensated for annual variation")
plt.legend()